<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/alignn_jarvis_leaderboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook shows how to add benchmarks to [JARVIS-Leaderboard](https://pages.nist.gov/jarvis_leaderboard/) using [alignn](https://github.com/usnistgov/alignn) for [exfoliation_energy](https://pages.nist.gov/jarvis_leaderboard/AI/SinglePropertyPrediction/dft_3d_exfoliation_energy/) task.

Install alignn

In [1]:
!pip install --pre dgl -f https://data.dgl.ai/wheels/cu117/repo.html
!pip install --pre dglgo -f https://data.dgl.ai/wheels-test/repo.html
!pip install alignn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/cu117/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 MB 9.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

First, let's install jarvis_leaderboard

In [2]:
pip install  jarvis_leaderboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: markdown
    Found existing installation: Markdown 3.4.3
    Uninstalling Markdown-3.4.3:
      Successfully uninstalled Markdown-3.4.3
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency re

Check if you can import libraries

In [3]:
import jarvis_leaderboard

In [4]:
import alignn

Let's populate a dataset (AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae). Files will be saved in the output_path i.e. 'Out' directory

In [5]:
!jarvis_populate_data.py --benchmark_file AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae --output_path=Out

benchmark_file AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae
dataset dft_3d
output_path Out
property exfoliation_energy
method AI
task SinglePropertyPrediction
id_tag jid
dataset file to be used /usr/local/lib/python3.9/dist-packages/jarvis_leaderboard/dataset/AI/SinglePropertyPrediction/dft_3d_exfoliation_energy.json.zip
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
100% 40.8M/40.8M [00:02<00:00, 17.2MiB/s]
Loading the zipfile...
Loading completed.
number of training samples 650
number of validation samples 81
number of test samples 81


In [6]:
!rm -r exfoliation_en*

rm: cannot remove 'exfoliation_en*': No such file or directory


In [7]:
ls Out/dataset_info.json

Out/dataset_info.json


In [8]:
from jarvis.db.jsonutils import loadjson
dataset_info = loadjson('Out/dataset_info.json')

In [9]:
dataset_info

{'benchmark_file': 'AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae',
 'dataset': 'dft_3d',
 'output_path': 'Out',
 'prop': 'exfoliation_energy',
 'methods': 'AI',
 'id_tag': 'jid',
 'n_train': 650,
 'n_val': 81,
 'n_test': 81}

In [10]:
n_train = dataset_info['n_train']
n_val = dataset_info['n_val']
n_test = dataset_info['n_test']

Download an example config file.

In [11]:
!wget https://raw.githubusercontent.com/usnistgov/alignn/main/alignn/examples/sample_data/config_example.json

--2023-04-26 05:27:13--  https://raw.githubusercontent.com/usnistgov/alignn/main/alignn/examples/sample_data/config_example.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1411 (1.4K) [text/plain]
Saving to: ‘config_example.json’

config_example.json 100%[===================>]   1.38K  --.-KB/s    in 0s      

2023-04-26 05:27:13 (12.9 MB/s) - ‘config_example.json’ saved [1411/1411]



In [12]:
from jarvis.db.jsonutils import loadjson
config = loadjson('config_example.json')

In [13]:
config

{'version': '112bbedebdaecf59fb18e11c929080fb2f358246',
 'dataset': 'user_data',
 'target': 'target',
 'atom_features': 'cgcnn',
 'neighbor_strategy': 'k-nearest',
 'id_tag': 'jid',
 'random_seed': 123,
 'classification_threshold': None,
 'n_val': None,
 'n_test': None,
 'n_train': None,
 'train_ratio': 0.8,
 'val_ratio': 0.1,
 'test_ratio': 0.1,
 'target_multiplication_factor': None,
 'epochs': 3,
 'batch_size': 2,
 'weight_decay': 1e-05,
 'learning_rate': 0.001,
 'filename': 'sample',
 'warmup_steps': 2000,
 'criterion': 'mse',
 'optimizer': 'adamw',
 'scheduler': 'onecycle',
 'pin_memory': False,
 'save_dataloader': False,
 'write_checkpoint': True,
 'write_predictions': True,
 'store_outputs': True,
 'progress': True,
 'log_tensorboard': False,
 'standard_scalar_and_pca': False,
 'use_canonize': True,
 'num_workers': 0,
 'cutoff': 8.0,
 'max_neighbors': 12,
 'keep_data_order': True,
 'model': {'name': 'alignn',
  'alignn_layers': 4,
  'gcn_layers': 4,
  'atom_input_features': 92,
 

In [15]:
config['n_train'] = n_train
config['n_val'] = n_val
config['n_test'] = n_test
config['epochs'] = 10
config['batch_size'] = 10

In [16]:
from jarvis.db.jsonutils import dumpjson
dumpjson(data=config,filename='tmp_config.json')

In [17]:
%%time
!train_folder.py --root_dir "Out" --config "tmp_config.json" --output_dir="temp"

DGL backend not selected or invalid.  Assuming PyTorch for now.
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
fatal: not a git repository (or any of the parent directories): .git
MAX val: 948.93
MIN val: 0.03
MAD: 62.662647995583484
Baseline MAE: 76.21933542260209
data range 946.44 0.03
  0% 0/650 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/alignn/graphs.py:237: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  g.ndata["lattice_mat"] = torch.tensor(
100% 650/650 [00:14<00:00, 43.48it/s]
df                                                  atoms  ...  target
0    {'lattice_mat': [[1.6788427119082352, -2.90784...  ...  

We choose one of the SinglePropertyPrediction tasks from the [leaderboard](https://pages.nist.gov/jarvis_leaderboard/AI/SinglePropertyPrediction/). See other examples in the [alignn_model](https://github.com/usnistgov/jarvis_leaderboard/tree/main/jarvis_leaderboard/benchmarks/alignn_model) benchmark folder

Prepare the csv file with six components in the filename.

In [18]:
import os
cwd=os.getcwd()
os.makedirs('alignn_test')
os.chdir('alignn_test')

In [19]:
!cp ../temp/prediction_results_test_set.csv ./AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae.csv 

In [20]:
!zip AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae.csv.zip AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae.csv 

  adding: AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae.csv (deflated 59%)


In [21]:
!wget https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/metadata.json

--2023-04-26 05:29:26--  https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/metadata.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391 [text/plain]
Saving to: ‘metadata.json’

metadata.json       100%[===================>]     391  --.-KB/s    in 0s      

2023-04-26 05:29:26 (23.4 MB/s) - ‘metadata.json’ saved [391/391]



In [22]:
!rm AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae.csv 

In [23]:
from jarvis.db.jsonutils import loadjson,dumpjson
meta = loadjson('metadata.json')

In [24]:
meta

{'model_name': 'ALIGNN',
 'project_url': 'https://www.nature.com/articles/s41524-021-00650-1',
 'date_submitted': '01-14-2023',
 'author_email': 'knc6@nist.gov',
 'database_version': '12-12-2022',
 'team_name': 'JARVIS',
 'time_taken_seconds': 458.17,
 'language': 'python',
 'os': 'linux',
 'software_used': 'jarvis-tools,numpy,scipy,torch,alignn',
 'harware_used': 'nisaba-cluster at NIST, V100 Tesla GPU'}

In [25]:
meta['model_name'] = 'alignn_test'

# TODO: Add other details
dumpjson(data=meta,filename='metadata.json')

In [26]:
!wget https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/run.sh

--2023-04-26 05:30:01--  https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/run.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54 [text/plain]
Saving to: ‘run.sh’

run.sh              100%[===================>]      54  --.-KB/s    in 0s      

2023-04-26 05:30:01 (3.63 MB/s) - ‘run.sh’ saved [54/54]



In [27]:
os.chdir(cwd)

1. Replace knc6 by your own GitHub username.
2. You might have to add your [GitHub password/token](https://github.com/settings/tokens) twice at the command prompt after you run the following command.
3. This will create a new pull request at the parent repo: https://github.com/usnistgov/jarvis_leaderboard/pulls
4. After an admin approves the contribution, the benchmark will be added to the main website.


In [29]:
!jarvis_upload.py --your_benchmark_directory  alignn_test --github_username knc6

For help: jarvis_upload.py -h
Using GitHub username knc6
Forked_url https://github.com/knc6/jarvis_leaderboard
response <Response [200]>
Note: adding to existing directory.
Cpying files cp -r alignn_test jarvis_leaderboard/jarvis_leaderboard/benchmarks/
List files ls ./jarvis_leaderboard/benchmarks/alignn_test
AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae.csv.zip	run.sh
metadata.json
Git add dir git add ./jarvis_leaderboard/benchmarks/alignn_test/*
Git commit git commit -m 'Adding benchmark by knc6_alignn_test.'

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@04db784a046a.(none)')
Push git push
fatal: could not read Username for 'https://github.com': No such device or address
python jarvis_leaderboard/rebuild.py
Rebuilding web:
jarv

The above script automatically:

   a) forks the jarvis_leaderboard repo from `https://github.com/usnistgov/jarvis_leaderboard` to your GitHub account,

   b) clones the forked jarvis_leaderboard from GitHub,  

   c) copies the `test_bench` folder in the `jarvis_leaderboard/benchmarks` folder,

   d) check if adding your benchmarks breaks the workflow using `rebuild.py` script,

   e) `git add`, `git commit`, and `git push` to the forked repo,

   f) create a pull request to the main usnistgov repo,

   g) an admin will approve the benchmark and new pages will be created.


In [31]:
pip freeze

absl-py==1.4.0
alabaster==0.7.13
albumentations==1.2.1
alignn==2023.4.1
altair==4.2.2
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.15.1
ase==3.22.1
astropy==5.2.2
astunparse==1.6.3
attrs==23.1.0
audioread==3.0.0
autograd==1.5
autopep8==2.0.2
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
blosc2==2.0.0
bokeh==2.4.3
branca==0.6.0
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorama==0.4.6
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==40.0.2
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.34
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.1.1
dbus-python==1.2.16
debu